# AutoGraph的使用规范|

## 一，Autograph编码规范总结


* 1，被@tf.function修饰的函数应尽可能使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print，使用tf.range而不是range，使用tf.constant(True)而不是True.

* 2，避免在@tf.function修饰的函数内部定义tf.Variable. 

* 3，被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量。

In [1]:
import tensorflow as tf
import numpy as np

## 二、Autograph编码规范解析

### 1.被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数

In [9]:
@tf.function
def np_random():
    tf.print(np.random.randn(3, 3))
    
@tf.function
def tf_random():
    tf.print(tf.random.normal((3,3)))

In [10]:
# np_random每次执行的结果都是一样的
np_random()
np_random()

array([[ 1.96233833,  0.34878665, -0.71457562],
       [-0.38389489,  0.27789599, -1.32623745],
       [-0.8100119 ,  1.54516684, -1.11343351]])
array([[ 1.96233833,  0.34878665, -0.71457562],
       [-0.38389489,  0.27789599, -1.32623745],
       [-0.8100119 ,  1.54516684, -1.11343351]])


In [11]:
# tf_random每次执行会生成不一样的结果
tf_random()
tf_random()

[[0.337754458 1.43786895 0.296255261]
 [-0.438719809 0.673409104 0.148053885]
 [0.794883549 -0.891180873 0.715625465]]
[[0.749949872 -0.421394885 -0.59615463]
 [0.787236154 -1.725281 1.31373835]
 [0.246683 0.0395259783 -1.04545224]]


### 2.避免在@tf.function修饰的函数内部定义tf.Variable

In [13]:
x = tf.Variable(1.0, dtype = tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return x

outer_var()
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [15]:
# 内部定义变量， 执行将会报错
@tf.function
def inner_var():
    x = tf.Variable(1.0, dtype = tf.float32)
    x.assign_add(1.0)
    return x

inner_var()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ValueError: in user code:

    <ipython-input-15-dbb24a131247>:4 inner_var  *
        x = tf.Variable(1.0, dtype = tf.float32)
    /root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/variables.py:261 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    /root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/variables.py:255 _variable_v2_call
        shape=shape)
    /root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/variables.py:66 getter
        return captured_getter(captured_previous, **kwargs)
    /root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py:511 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


### 3.被@tf.function修饰的函数不可以修改该函数外部的Python列表或字典数据结构

In [20]:
tensor_list = []

@tf.function
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(1.0))
append_tensor(tf.constant(2.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]
